<h1 style="color: orange;">NASA API</h1>

**Example:**
1. Fetching one new image per class (e.g., "galaxy") using the NASA API.
2. Avoiding duplicate downloads by maintaining a record of downloaded URLs.
3. Scalability to `.py` scripts.

In [27]:
import requests
import os
import glob
import matplotlib.pyplot as plt

In [ ]:
# Save your Key somewhere
API_KEY = ""

In [12]:
if not API_KEY:
    raise ValueError("No NASA API key found. Please set the NASA_API_KEY environment variable.")

<h1 style="color: orange;">Function to Fetch & Save</h1>

This function will:
- Query the NASA Image Library for a specific class (e.g., "galaxy").
- Save the first image from the results.

In [18]:
# Directory to save images and data location
BASE_URL = "https://images-api.nasa.gov/search"
os.makedirs(SAVE_DIR, exist_ok=True)

In [23]:
def fetch_images(query, save_dir, downloaded_urls, max_images=5):
    """
    Fetch and save multiple new images for the given query.

    Parameters:
        - query: The class to search for (e.g., "galaxy").
        - save_dir: Directory to save the images.
        - downloaded_urls: Set of previously downloaded URLs to avoid duplicates.
        - max_images: Maximum number of images to download for the query.
    """
    page = 1
    downloaded_count = 0

    while downloaded_count < max_images:
        params = {
            "q": query,
            "media_type": "image",
            "page": page
        }
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()
        data = response.json()

        # Get items from the current page
        items = data.get("collection", {}).get("items", [])
        if not items:
            print(f"No more results for query: {query}")
            break

        for item in items:
            links = item.get("links", [])
            for link in links:
                image_url = link.get("href")
                if image_url and image_url not in downloaded_urls:
                    # Download and save the image
                    title = item["data"][0]["title"].replace(' ', '_').replace('/', '_')
                    image_name = f"{title}.jpg"
                    image_path = os.path.join(save_dir, image_name)

                    try:
                        img_data = requests.get(image_url).content
                        with open(image_path, "wb") as img_file:
                            img_file.write(img_data)
                        downloaded_urls.add(image_url)  # Add to the set of downloaded URLs
                        downloaded_count += 1
                        print(f"Image saved: {image_name}")
                        if downloaded_count >= max_images:
                            return  # Stop after downloading the maximum number of images
                    except Exception as e:
                        print(f"Failed to save the image: {e}")
        page += 1
    print(f"Downloaded {downloaded_count} images for query: {query}")

<h1 style="color: orange;">Fetch and Save an Image</h1>

We will now fetch and save one image for the specified class.

In [20]:
# Initialize save directory and downloaded URLs
SAVE_DIR = "./nasa_images"
os.makedirs(SAVE_DIR, exist_ok=True)
downloaded_urls = set()  # Store downloaded image URLs

In [21]:
# Specify a query and fetch one new image
class_name = "galaxy"
fetch_one_image(class_name, SAVE_DIR, downloaded_urls)

Image saved: Andromeda_Galaxy.jpg


True

Now, we fetch one new image for each class, ensuring no duplicates.

In [22]:
# Fetch one new image for each class
classes = ["galaxy", "nebula", "planet", "star", "comet", "asteroid", "black hole"]

for class_name in classes:
    print(f"Fetching one image for class: {class_name}")
    success = fetch_one_image(class_name, SAVE_DIR, downloaded_urls)
    if not success:
        print(f"No new images available for class: {class_name}")

Fetching one image for class: galaxy
Image saved: Galaxy_NGC5474.jpg
Fetching one image for class: nebula
Image saved: Weighing_in_on_the_Dumbbell_Nebula.jpg
Fetching one image for class: planet
Image saved: Proposed_Missions_-_Terrestrial_Planet_Finder.jpg
Fetching one image for class: star
Image saved: Hubble_Observes_One-of-a-Kind_Star_Nicknamed_‘Nasty’.jpg
Fetching one image for class: comet
Image saved: Rosetta_at_Comet.jpg
Fetching one image for class: asteroid
Image saved: Asteroid_Lithograph_of_Eros_and_Other_Asteroids.jpg
Fetching one image for class: black hole
Image saved: Behemoth_Black_Hole_Found_in_an_Unlikely_Place.jpg
